<a href="https://colab.research.google.com/github/li0217codeninja/time-seq-learning/blob/main/timeseriesTransformer.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!python --version

Python 3.10.12


In [ ]:
import torch
from torch import nn
from torch.utils.data import DataLoader, Dataset
import numpy as np

# Timeseries PyTorch Transformer

In [ ]:
class SequenceDataset(torch.utils.data.Dataset):
    def __init__(self, seq_length):
        self.seq_length = seq_length

    def __len__(self):
        return 10000  # Replace with actual dataset size

    def __getitem__(self, idx):
        # Generate a sequence number
        seq_num = torch.tensor(idx)

        # Generate a sequence of numbers based on seq_num
        seq_data = torch.randn(self.seq_length) * seq_num

        return seq_num, seq_data

def collate_fn(samples):
    seq_nums, seq_data = zip(*samples)
    # Pad or truncate sequences to a fixed length (optional)
    # ...
    return torch.stack(seq_nums), torch.stack(seq_data)

train_loader = DataLoader(
    SequenceDataset(seq_length=32),
    batch_size=32,
    shuffle=True,
    collate_fn=collate_fn,
)


In [ ]:
transformer_model = nn.Transformer(nhead=16)
src = torch.rand(10, 32, 512)
tgt = torch.rand(20, 32, 512)
out = transformer_model(src, tgt)

In [ ]:
loss_fn = torch.nn.MSELoss()
optimizer = torch.optim.Adam(transformer_model.parameters())

epochs = 10
for epoch in range(epochs):
  for batch_idx, (data, target) in enumerate(train_loader):
    # Forward pass
    output = transformer_model(data,target)
    # Loss calculation
    loss = loss_fn(output, target)
    # Backward pass and parameter update
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()


In [ ]:
transformer_model.parameters()

<generator object Module.parameters at 0x7a4ca5ac67a0>

## Custom train loop with LSTM

1.   understand the bi-direction argument in lstm
2.   how to dignose if any issue with model trainining
3.   one step prediction, can we do multi-step forcast? is it just a for loop.





In [ ]:
class RNNmodel(nn.Module):
  def __init__(self, input_size, hidden_size,  num_features):
    super(RNNmodel,self).__init__() #todo check super()
    self.lstm = nn.LSTM(input_size, hidden_size,  batch_first=True)
    self.dense = nn.Linear(hidden_size, num_features)

  def forward(self, x):
    h, _ = self.lstm(x) # output, (h_n, c_n)
    output = self.dense(h[:,-1,:])
    return output


In [ ]:
class DummyDataset(Dataset):
  def __init__(self,  data, sequence_len):
    # 100 sequences, 10 time steps, 2 features per variable, 3 variables
    self.data = data
    self.sequence_length = sequence_len

  def __len__(self):
    return len(self.data) - self.sequence_length

  def __getitem__(self, idx):
        x = torch.tensor(self.data[idx:idx + self.sequence_length, :], dtype=torch.float32)
        y = torch.tensor(self.data[idx + self.sequence_length, :], dtype=torch.float32)
        return x, y

In [ ]:
# Generate dummy data
np.random.seed(42)
num_samples = 1000
sequence_length = 10
num_features = 3

data = np.random.randn(num_samples, num_features)

# Create the model, dataset and data loader
input_size = num_features
hidden_size = 50
output_size = num_features

model = RNNmodel(input_size, hidden_size, output_size)
dataset = DummyDataset(data, sequence_length)
dataloader = DataLoader(dataset, batch_size=32, shuffle=True, drop_last=True)
optimizer = torch.optim.SGD(model.parameters(), lr=0.001, momentum=0.9)

for epoch in range(100):
  for batch_x, batch_y in dataloader:
        # Training logic goes here
        # e.g., forward pass, loss computation, backward pass, and optimization
        outputs = model(batch_x)
        loss = nn.MSELoss()(outputs, batch_y)

        # Your training steps here
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

  print(f'Epoch {epoch + 1}, Batch Loss: {loss.item()}')



In [ ]:
# Assuming model is your PyTorch model
initial_params = sum(p.numel() for p in model.parameters())
print("Initial number of parameters:", initial_params)

# Check if any parameters have been updated
updated_params = sum(p.numel() for p in model.parameters())
if updated_params > initial_params:
    print("The model has been trained.")
else:
    print("The model has not been trained.")

Initial number of parameters: 11153
The model has not been trained.


In [ ]:
# Inference
# After training, you can use the trained model for predictions
test_sequence = torch.tensor(np.random.randn(11, num_features), dtype=torch.float32).unsqueeze(0)  # Add batch dimension
test_sequence
prediction = model(test_sequence)

mse = nn.MSELoss()(prediction, test_sequence[:,-1,:])
mse

tensor(0.9112, grad_fn=<MseLossBackward0>)

In [ ]:
# Print model's state_dict
print("Model's state_dict:")
for param_tensor in model.state_dict():
    print(param_tensor, "\t", model.state_dict()[param_tensor].size())

print()

# Print optimizer's state_dict
print("Optimizer's state_dict:")
for var_name in optimizer.state_dict():
    print(var_name, "\t", optimizer.state_dict()[var_name])


# Custom Dataloader

ref: [Biswajit -  data loader](https://biswajitsahoo1111.github.io/post/reading-multiple-csv-files-in-pytorch/)

## Generate random files

In [ ]:
import numpy as np
import os
import glob
np.random.seed(42)

In [ ]:
np.random.rand(1024, ).shape

(1024,)

In [ ]:
def create_rand_csv_files(classes, num_files):
  try:
    os.mkdir('./random_data')
  except:
    pass

  for c in classes:
    for i in range(num_files):
      data = np.random.rand(1024, )
      file_name = './random_data/' + eval('c') + '_' + '{0:03}'.format(i+1) + '.csv'
      np.savetxt(file_name, data, delimiter=',', header='data', comments='')

create_rand_csv_files(['type_a', 'type_b', 'type_c'], 10)

## Write custom dataloader

In [ ]:
import pandas as pd
import re
import torch
from torch.utils.data import Dataset
print('PyTorch Version: ', torch.__version__)
import glob

PyTorch Version:  2.1.0+cu121


In [ ]:
class CustomDataset(Dataset):
  def __init__(self, filenames, batch_size):
    # filenames: a list of strings that contain all file names
    # batch_size: determins the number of files that we read in chunk
    self.filenames = filenames
    self.batch_size = batch_size

  def __len__(self):
    return int(np.ceil(len(self.filenames)/float(self.batch_size))) #number batch/chunk of files

  def __getitem__(self, idx):
    #idx: index of the chunk
    # in this method, we do all preprocessing of the data

    # first read data in a chunk and preprocess -> extract data and labels
    batch_x = self.filenames[idx * self.batch_size : (idx + 1) * self.batch_size]

    data = []
    labels = []
    label_classes = ['type_a','type_b','type_c']

    for file in batch_x:
      temp = pd.read_csv(file)

      # preprocess step - customize this to extract and reshape feautres
      data.append(temp.values.reshape(32,32,1))


      # customize this to extract labels
      #breakpoint()
      pattern = eval("file[14:20]")
      for j in range(len(label_classes)):
        #breakpoint()
        if re.match(pattern, label_classes[j]):
          labels.append(j)

    # pytoch channel fisrt conventions. check conventions
    data = np.asarray(data).reshape(-1, 1, 32, 32)
    labels = np.asarray(labels)

    # the following condition is actually needed in pytorch to prevent infinite loop
    # verify if this is true by removing the cond
    if idx == self.__len__():
      raise IndexError

    return data, labels


files = glob.glob("./random_data/*")
print(files[:5])

check_dataset = CustomDataset(filenames= files, batch_size=10)
check_dataset.__len__()

['./random_data/type_a_001.csv', './random_data/type_c_003.csv', './random_data/type_b_010.csv', './random_data/type_b_003.csv', './random_data/type_c_009.csv']


3

In [ ]:
for i, (data, labels) in enumerate(check_dataset):
  print(data.shape, labels.shape)
  print(labels)
  if i==3: break

(10, 1, 32, 32) (10,)
[0 2 1 1 2 0 0 0 0 1]
(10, 1, 32, 32) (10,)
[1 0 2 0 2 1 1 2 1 1]
(10, 1, 32, 32) (10,)
[2 0 1 1 0 0 2 2 2 2]


# Time series Transformer Implementation
reference:
https://towardsdatascience.com/how-to-make-a-pytorch-transformer-for-time-series-forecasting-69e073d4061e

shell-gpt notes
https://github.com/TheR1D/shell_gpt


notes:
1. position encoder: supports multi-variates
2. checkout implementation of nn.TransformerEncoderLayer, nn.TransformerDecoderLayer
3. what is src_mask and tgt_mask

